In [3]:
from propositional_state_logic import *
from utils import *
from collections import defaultdict
import itertools
from copy import deepcopy

In [4]:
p = Problem()

# Define the variables above. Returns a Variable object.
F1 = p.add_variable('F1', type='binary')
P1 = p.add_variable('P1', type='binary')
V1 = p.add_variable('V1', type='binary')
P2 = p.add_variable('P2', type='binary')
V2 = p.add_variable('V2', type='binary')
PV2 = p.add_variable('PV2', type='binary')
V3 = p.add_variable('V3', type='binary')
PV3 = p.add_variable('PV3', type='binary')
P3 = p.add_variable('P3', type='binary')
R1 = p.add_variable('R1', type='binary')
T1 = p.add_variable('T1', type='binary')

# Add the word problem constraints.
p.add_constraint('F1 => P1')
p.add_constraint('~F1 => ~P1')

p.add_constraint('V1 & P1 => P2')
p.add_constraint('V1 & ~P1 => ~P2')
p.add_constraint('~V1 => ~P2')

p.add_constraint('V2 & P2 => PV2')
p.add_constraint('V2 & ~P2 => ~PV2')
p.add_constraint('~V2 => ~PV2')

p.add_constraint('V3 & P2 => PV3')
p.add_constraint('V3 & ~P2 => ~PV3')
p.add_constraint('~V3 => ~PV3')

p.add_constraint('~PV2 & ~PV3 => ~P3')
p.add_constraint('PV2 => P3')
p.add_constraint('PV3 => P3')

p.add_constraint('R1 & P3 => T1')
p.add_constraint('R1 & ~P3 => ~T1')
p.add_constraint('~R1 => ~T1')

# Prints out constraints nicely in LaTeX, so you can check them.
display_constraints(p)

Constraints:


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [5]:
sat = SATSolver(p)
# Make an assignment to some variables; variable.get_assignment(value) returns an Assignment.
assignment = frozenset([F1.get_assignment('True'), P1.get_assignment('False')])
sat.check_consistency(assignment)

(False, None, frozenset())

In [6]:
Vs = ['T1', 'R1', 'P3']
Vs

['T1', 'R1', 'P3']

In [7]:
def findsubsets(s, n):
    return (list(tup) for tup in itertools.combinations(s, n))

In [8]:
def get_nodes(variable_list):
    nodes = []

    for i in range(len(variable_list)+1):
        for n in findsubsets(variable_list, i):
            nodes.append(n)

    return nodes

In [9]:
all_domains = []

for v in Vs:
    var = p.get_variable(v)
    for a in list(var.domain):
        all_domains.append(var.get_assignment(a))

all_domains

[¬T1, T1, ¬R1, R1, ¬P3, P3]

In [10]:
full_set = get_nodes(all_domains)
reduced_set = []

for v in full_set:
    if len(v) == len(set([a.var for a in v])):
        #reduced_set.append([a.__str__() for a in v])
        reduced_set.append(v)

reduced_set


[[],
 [¬T1],
 [T1],
 [¬R1],
 [R1],
 [¬P3],
 [P3],
 [¬T1, ¬R1],
 [¬T1, R1],
 [¬T1, ¬P3],
 [¬T1, P3],
 [T1, ¬R1],
 [T1, R1],
 [T1, ¬P3],
 [T1, P3],
 [¬R1, ¬P3],
 [¬R1, P3],
 [R1, ¬P3],
 [R1, P3],
 [¬T1, ¬R1, ¬P3],
 [¬T1, ¬R1, P3],
 [¬T1, R1, ¬P3],
 [¬T1, R1, P3],
 [T1, ¬R1, ¬P3],
 [T1, ¬R1, P3],
 [T1, R1, ¬P3],
 [T1, R1, P3]]

In [11]:
def list_tree(nodes, n):
    new_nodes = deepcopy(nodes)
    if n in new_nodes:
        new_nodes.remove(n)

    a = []
    a.append(tuple(n))

    if new_nodes == [] or [j for j in new_nodes if n != j and [v.__str__() for v in n] == [v.__str__() for v in j[:len(n)]] and len(j) == len(n)+1] == []:
        a.append([])
        
    else:
        a.append([list_tree([k for k in new_nodes if k != j], j) for j in new_nodes if n != j and [v.__str__() for v in n] == [v.__str__() for v in j[:len(n)]] and len(j) == len(n)+1])
    
    return a

In [12]:
tree_1 = list_tree(reduced_set, [])
tree_1

[(),
 [[(¬T1,),
   [[(¬T1, ¬R1), [[(¬T1, ¬R1, ¬P3), []], [(¬T1, ¬R1, P3), []]]],
    [(¬T1, R1), [[(¬T1, R1, ¬P3), []], [(¬T1, R1, P3), []]]],
    [(¬T1, ¬P3), []],
    [(¬T1, P3), []]]],
  [(T1,),
   [[(T1, ¬R1), [[(T1, ¬R1, ¬P3), []], [(T1, ¬R1, P3), []]]],
    [(T1, R1), [[(T1, R1, ¬P3), []], [(T1, R1, P3), []]]],
    [(T1, ¬P3), []],
    [(T1, P3), []]]],
  [(¬R1,), [[(¬R1, ¬P3), []], [(¬R1, P3), []]]],
  [(R1,), [[(R1, ¬P3), []], [(R1, P3), []]]],
  [(¬P3,), []],
  [(P3,), []]]]

In [13]:
def list_tree(nodes, n):
    new_nodes = deepcopy(nodes)
    if n in new_nodes:
        new_nodes.remove(n)

    a = []
    a.append(n)

    if new_nodes == [] or [j for j in new_nodes if n != j and [v.__str__() for v in n] == [v.__str__() for v in j[:len(n)]] and len(j) == len(n)+1] == []:
        a.append([])
        
    else:
        a.append([list_tree([k for k in new_nodes if k != j], j) for j in new_nodes if n != j and [v.__str__() for v in n] == [v.__str__() for v in j[:len(n)]] and len(j) == len(n)+1])
    
    return a

In [14]:
tree_1 = list_tree(reduced_set, [])
tree_1

[[],
 [[[¬T1],
   [[[¬T1, ¬R1], [[[¬T1, ¬R1, ¬P3], []], [[¬T1, ¬R1, P3], []]]],
    [[¬T1, R1], [[[¬T1, R1, ¬P3], []], [[¬T1, R1, P3], []]]],
    [[¬T1, ¬P3], []],
    [[¬T1, P3], []]]],
  [[T1],
   [[[T1, ¬R1], [[[T1, ¬R1, ¬P3], []], [[T1, ¬R1, P3], []]]],
    [[T1, R1], [[[T1, R1, ¬P3], []], [[T1, R1, P3], []]]],
    [[T1, ¬P3], []],
    [[T1, P3], []]]],
  [[¬R1], [[[¬R1, ¬P3], []], [[¬R1, P3], []]]],
  [[R1], [[[R1, ¬P3], []], [[R1, P3], []]]],
  [[¬P3], []],
  [[P3], []]]]

In [110]:
def traverse(o):
    if o != []:
        if isinstance(o, list) and len(o) > 1:
            if not isinstance(o[0], list):
                yield o
            else:
                for value in o:
                    #print('value',value, isinstance(value, list))
                    for subvalue in traverse(value):
                        yield subvalue
        else:
            yield o

In [111]:
for value in traverse(tree_1):
    print(value)

[¬T1]
[¬T1, ¬R1]
[¬T1, ¬R1, ¬P3]
[¬T1, ¬R1, P3]
[¬T1, R1]
[¬T1, R1, ¬P3]
[¬T1, R1, P3]
[¬T1, ¬P3]
[¬T1, P3]
[T1]
[T1, ¬R1]
[T1, ¬R1, ¬P3]
[T1, ¬R1, P3]
[T1, R1]
[T1, R1, ¬P3]
[T1, R1, P3]
[T1, ¬P3]
[T1, P3]
[¬R1]
[¬R1, ¬P3]
[¬R1, P3]
[R1]
[R1, ¬P3]
[R1, P3]
[¬P3]
[P3]


In [ ]:
def traverse(data):
    print ' ' * traverse.level + data['text']
    for kid in data['kids']:
        traverse.level += 1
        traverse(kid)
        traverse.level -= 1

if __name__ == '__main__':
    traverse.level = 1

traverse(tree_1)